# Environment Setup

In [2]:
!/home/jjian03/anaconda3/bin/pip install singleton_decorator
!/home/jjian03/anaconda3/bin/pip install pyspark

In [3]:
import sys
sys.path.insert(0, '/home/jjian03/anaconda3/lib/python3.7/site-packages')

In [4]:
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2-2.3.0.cloudera3-1.cdh5.13.3.p0.458809/lib/spark2/')

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext


try:
    print(spark.version)
except NameError as e:
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.\
        config('spark.app.name', 'mongodb_migration').\
        config('spark.dynamicAllocation.enabled','true').\
        config('spark.dynamicAllocation.maxExecutors','50').\
        config('spark.dynamicAllocation.executorIdleTimeout','30s').\
        config('spark.driver.maxResultSize', '8g').\
        config('spark.driver.memory', '50g').\
        config('spark.executor.memory', '4g').\
        config('spark.task.maxFailures', '3').\
        config('spark.yarn.am.memory', '8g').\
        config('spark.yarn.max.executor.failures', '3').\
        config('spark.kryoserializer.buffer.max','1024m').\
        getOrCreate()
    print(spark.version)
sc = spark.sparkContext
spark_sql = SQLContext(sc)

2.3.0.cloudera3


In [7]:
from datetime import datetime
import gc


def load_dataset(spark, path, name):
    spark.read.parquet(path).registerTempTable(name)    

class Formatter:
    @staticmethod
    def get_timestamp(format="%Y/%m/%d %H:%M:%S"):
        return str((datetime.now().strftime(format)))

def watch():
    load_dataset(spark, '/user/jjian03/WebResourceQuality.parquet', 'WebResourceQuality')

    cnt = spark_sql.sql('''
        SELECT count(id) as cnt
        FROM WebResourceQuality
    ''').toPandas().cnt[0]
    print(f'{Formatter.get_timestamp()} - {cnt}')
    gc.collect()

In [12]:
import time

while True:
    watch()
    time.sleep(300)

2020/06/17 11:00:26 - 1989579
2020/06/17 11:06:18 - 1999575
2020/06/17 11:11:56 - 2004921
2020/06/17 11:17:42 - 2004921
2020/06/17 11:23:29 - 2004921
2020/06/17 11:30:40 - 2004921
2020/06/17 11:36:08 - 2004921
2020/06/17 11:41:26 - 2004921
2020/06/17 11:46:59 - 2004921


KeyboardInterrupt: 

In [11]:
load_dataset(spark, '/user/jjian03/WebResourceQuality.parquet', 'WebResourceQuality')
load_dataset(spark, '/datasets/MAG_20200403/MAG_Azure_Parquet/mag_parquet/Papers.parquet', 'Paper')
load_dataset(spark, '/datasets/MAG_20200403/MAG_Azure_Parquet/mag_parquet/PaperReferences.parquet', 'PaperReference')
load_dataset(spark, '/datasets/MAG_20200403/MAG_Azure_Parquet/mag_parquet/AuthorAffiliation.parquet', 'PaperAuthorAffiliation')
# load_dataset(spark, '/datasets/MAG_20200403/MAG_Azure_Parquet/mag_parquet/Authors.parquet', 'Author')
# load_dataset(spark, '/datasets/MAG_20200403/MAG_Azure_Parquet/mag_parquet/Affiliations.parquet', 'Affiliation')

spark_sql.sql('''
        SELECT *
        FROM WebResourceQuality
    ''').limit(1).toPandas().columns

Index(['id', 'url', 'actual_scrape_url', 'first_appear',
       'first_available_timestamp', 'last_available_timestamp', 'header',
       'html_text', 'comment', 'from_waybackmachine', 'http_status_code',
       'original_check_failure', 'original_check_error_log',
       'terminate_reason', 'terminate_reason_error_log', 'label'],
      dtype='object')